Import everything


In [22]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import numpy as np
from sklearn.feature_selection import SequentialFeatureSelector



In [23]:
df1 = pd.read_csv("/Users/talalkhan/Documents/Data Sets/train.csv")
df2 = pd.read_csv("/Users/talalkhan/Documents/Data Sets/test.csv")


In [24]:
'''#handle categoriacl values in both data sets 
df1 = pd.get_dummies(df1, columns=['ethnicity', 'gender', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem'])
df2 = pd.get_dummies(df2, columns=['ethnicity', 'gender', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem'])
'''
#do onehot encoding for categorical columns
df1 = pd.get_dummies(df1)
df2 = pd.get_dummies(df2)


'''
#another way to handle categorical data
# List of categorical columns to label encode
categorical_columns = ['ethnicity', 'gender', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']

# Initialize a LabelEncoder for each categorical column
label_encoders = {}
#label_encoders2 = {}

for column in categorical_columns:
    le = LabelEncoder()
    df1[column] = le.fit_transform(df1[column])
    label_encoders[column] = le

for column in categorical_columns:
    le2 = LabelEncoder()
    df2[column] = le2.fit_transform(df2[column])
    label_encoders[column] = le2


#df1.dtypes'''

"\n#another way to handle categorical data\n# List of categorical columns to label encode\ncategorical_columns = ['ethnicity', 'gender', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']\n\n# Initialize a LabelEncoder for each categorical column\nlabel_encoders = {}\n#label_encoders2 = {}\n\nfor column in categorical_columns:\n    le = LabelEncoder()\n    df1[column] = le.fit_transform(df1[column])\n    label_encoders[column] = le\n\nfor column in categorical_columns:\n    le2 = LabelEncoder()\n    df2[column] = le2.fit_transform(df2[column])\n    label_encoders[column] = le2\n\n\n#df1.dtypes"

In [25]:
'''#using simpleimputer to handle nan values
#df1
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df1.values)
imputed_data1 = imr.transform(df1.values)
#df2
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df2.values)
imputed_data2 = imr.transform(df2.values)
'''
#using KNNImputer to handle nan values
#df1
#'''
imr = KNNImputer(n_neighbors=2500, weights='uniform')
imr = imr.fit(df1.values)
imputed_data1 = imr.transform(df1.values)
#df2
imr = KNNImputer(n_neighbors=2500, weights='uniform')
imr = imr.fit(df2.values)
imputed_data2 = imr.transform(df2.values)
#'''
# convert the imputed NumPy array back into a Pandas DataFrame
df1 = pd.DataFrame(imputed_data1, columns=df1.columns)
df2 = pd.DataFrame(imputed_data2, columns=df2.columns)



In [26]:
#MinMaxscaling 
scaler = MinMaxScaler()
cols_to_scale = df1.columns[df1.columns != 'RecordID']
temp = df1.loc[:, ['RecordID']]
df1 = pd.DataFrame(scaler.fit_transform(df1[cols_to_scale]), columns=scaler.get_feature_names_out())
df1 = pd.concat([temp, df1], axis=1, join='inner')

cols_to_scale = df2.columns[df2.columns != 'RecordID']
temp = df2.loc[:, ['RecordID']]
df2 = pd.DataFrame(scaler.fit_transform(df2[cols_to_scale]), columns=scaler.get_feature_names_out())
df2 = pd.concat([temp, df2], axis=1, join='inner')

In [ ]:
# Calculate correlations with the 'hospital_death' column
correlations = df1.corr()['hospital_death']

# Display all correlations
for column, correlation in correlations.items():
    print(f"Correlation with {column}: {correlation}")

In [28]:
#drop colums with low correlation RecordID,encounter_id,patient_id,hospital_id,icu_id
df1 = df1.drop(['ventilated_apache','apache_4a_hospital_death_prob','icu_stay_type_readmit', 'apache_3j_bodysystem_Gynecological', 'apache_2_bodysystem_Undefined Diagnoses'], axis=1)
df2 = df2.drop(['ventilated_apache', 'apache_4a_hospital_death_prob','icu_stay_type_readmit', 'apache_3j_bodysystem_Gynecological', 'apache_2_bodysystem_Undefined Diagnoses'], axis=1)

'''threshold = 0.03  # Set your correlation threshold here

# Identify columns with correlation below the threshold
low_corr_columns = correlations[abs(correlations) < threshold].index
low_corr_columns = [col for col in low_corr_columns if col != 'RecordID']

# Drop the identified columns from df1
df1 = df1.drop(low_corr_columns, axis=1)
df2 = df2.drop(low_corr_columns, axis=1)
# Now df1 contains only columns with correlations above the threshold
'''

"threshold = 0.03  # Set your correlation threshold here\n\n# Identify columns with correlation below the threshold\nlow_corr_columns = correlations[abs(correlations) < threshold].index\nlow_corr_columns = [col for col in low_corr_columns if col != 'RecordID']\n\n# Drop the identified columns from df1\ndf1 = df1.drop(low_corr_columns, axis=1)\ndf2 = df2.drop(low_corr_columns, axis=1)\n# Now df1 contains only columns with correlations above the threshold\n"

In [29]:
X = df1.loc[:, df1.columns != 'hospital_death']
y = df1['hospital_death']

#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#create decision tree classifier
clf = DecisionTreeClassifier(random_state=1)

In [ ]:
'''
param_grid = {
    'max_depth': [5, 7, 9, 10],
    'min_samples_split': [5, 10, 7, 20],
    'min_samples_leaf': [50, 250, 550,600]
}


# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)
'''

In [31]:
#adjust hyperparameters
clf.set_params(max_depth=9, min_samples_split=7, min_samples_leaf=600)

#train model
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=9, min_samples_leaf=600, min_samples_split=7,
                       random_state=1)

In [20]:
sfs = SequentialFeatureSelector(clf, direction='forward',n_features_to_select=5, scoring='roc_auc')
sfs.fit(X, y)
print(sfs.get_feature_names_out())

['ventilated_apache' 'apache_4a_hospital_death_prob'
 'icu_stay_type_readmit' 'apache_3j_bodysystem_Gynecological'
 'apache_2_bodysystem_Undefined Diagnoses']


In [32]:
#make predictions
y_pred = clf.predict(X_test)
md_pred = clf.predict_proba(df2)

In [33]:
pred = md_pred[:,1]
#print mdpred up to 6 decimal places
print(md_pred[:,0].round(6))
#calculate accuracy
score = accuracy_score(y_test, y_pred)
print('Accuracy: %.3f' % score)

print(classification_report(y_test, y_pred))

[0.992495 0.370732 0.955923 ... 0.995058 0.983402 0.998601]
Accuracy: 0.919
              precision    recall  f1-score   support

         0.0       0.93      0.98      0.96      9101
         1.0       0.61      0.27      0.38       899

    accuracy                           0.92     10000
   macro avg       0.77      0.63      0.67     10000
weighted avg       0.90      0.92      0.90     10000



In [11]:
# Create a DataFrame for the results with RecordID and predicted probability of death
results_df = pd.DataFrame({'RecordID': df2['RecordID'] , 'ProbaDeath': pred})

# Save the results to a CSV file
results_df.to_csv('submission31_25253.csv', index=False)
